## ISE435 Final Project Code: Food Distribution Visualization

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json

#### Get geometry data for 34 NC counties served by Food Banks of Central and Eastern NC

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
st_code = "37"
counties['features'] = [STATE for STATE in counties['features'] if STATE['properties']['STATE'] == st_code]

#### Get specific county data & create empty column for FEMA designation

In [ ]:
df = pd.read_csv('NCFIPS.csv')
df['FEMA'] = np.zeros(34)
df.head()

#### Assign FEMA designation for each county

In [ ]:
IA = {'Durham'}
IADict = {i: 'Individual Assistance' for i in IA}

IPA = {'Chatham', 'Orange'}
IPADict = {i: 'Individual and Public Assistance' for i in IPA}

IPAAG = {'Brunswick', 'Carteret', 'Columbus', 'Craven', 'Duplin', 'Greene', 'Harnett', 'Johnston', 'Jones', 'Lee', 'Lenoir',
        'Moore', 'New Hanover', 'Onslow', 'Pamlico', 'Pender', 'Pitt', 'Richmond', 'Sampson', 'Scotland', 'Wayne', 'Wilson'}
IPAAGDict = {i: 'Individual and Public Assistance (Cat A-G)' for i in IPAAG}

noDes = {'Edgecombe', 'Franklin', 'Halifax', 'Nash', 'Vance', 'Wake', 'Warren'}
noDesDict = {i: 'No Designation' for i in noDes}

pubA = {'Granville', 'Person'}
pubADict = {i: 'Public Assistance' for i in pubA}


FEMADict = {**IADict, **IPADict, **IPAAGDict, **noDesDict, **pubADict}

#### Create df with necessary information for the visualization 

In [ ]:
countyData = df[['Branch ','COUNTY','Population', 'FIPS', 'FEMA']]
countyData['FEMA'] = countyData['COUNTY'].apply(lambda x: FEMADict[x])
countyData.head()

#### Read FBCENC daily data showing the difference in food distributed to 800 agencies between 2017 & 2018

In [ ]:
df2 = pd.read_csv('Daily.csv')
agencyData = df2[['Agency','Date','Latitude','Longitude', '2017', '2018']]
agencyData.head()

#### Get the difference from pounds of food distributed from 2017 to 2018

In [ ]:
diff = agencyData['2018'] - agencyData['2017']
absVal = diff.abs()

#### Add three columns: (1) Difference, (2) Absolute value to plot size of bubble on the visualization, (3) Amount to show More/Less

In [ ]:
agencyData['Difference'], agencyData['AbsVal'], agencyData['Amount'] = [diff, diff.abs(), diff.abs()]
agencyData

#### Update Amount column to show More/Less

In [ ]:
less = (agencyData.Difference<=0)
more = (agencyData.Difference>0)
agencyData.loc[less, 'Amount'] = 'Less'
agencyData.loc[more, 'Amount'] = 'More'
agencyData

#### Create scatter plot showing all 800 agencies pounds of food received between 20 August and 15 October

In [ ]:
fig = px.scatter_geo(agencyData, locationmode = 'USA-states', lon = 'Longitude', lat = 'Latitude',
                     color= 'Amount',
                     color_discrete_sequence= ["black", "red"],
                     size= 'AbsVal',
                     height = 600,
                     width = 950,
                     hover_name= 'Agency',
                     hover_data = {'AbsVal':False,
                                   'Difference':True,
                                   'Latitude':False,
                                   'Longitude':False},
                     animation_frame= 'Date',
                     )             

#### Create choropleth showing all the counties outlined, color-coded with their FEMA designation

In [ ]:
fig2 = px.choropleth(countyData, geojson=counties, locations= 'FIPS', featureidkey = 'id',
                    color = 'FEMA',
                    color_discrete_sequence= ["pink", "orange", "green", "white", "yellow"], 
                    scope = 'usa',
                    hover_name= 'COUNTY',
                    hover_data = {'FIPS':False,
                                  'Branch ':True,
                                  'COUNTY':True,
                                  'Population': True
                                    }
                    )     

#### Place fig2 on top of fig

In [ ]:
for i in range(0,5):
    fig.add_trace(fig2.data[i])

#### Update fig scale to show specific region in NC of interest to FBCENC  

In [ ]:
fig.update_geos(center=dict(lon=-78, lat=35.2),
                lataxis_range=[33,36], 
                lonaxis_range=[-79.5, -75]
               )

# Update fig layout & display
fig.update_layout(legend= dict(yanchor="top", y=0.99, xanchor="right", x=1.07, bordercolor="Black", borderwidth=1),
                  legend_title_text= "Amount Received and FEMA Classification", 
                  title={
                        'text': "Pounds of Food Received by FBCENC Agencies during Hurricane Florence",
                        'y':0.94,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'}
                 )
fig.show()